In [1]:
%load_ext autoreload
%autoreload 2

In [74]:
import logging

from constants.companies import get_company_by_code

company_code = 'AMK'
company = get_company_by_code(company_code=company_code)
company_id = company.company_id

In [ ]:
from catalog_connector import connection
df_truth = connection.sql(
    f"""
        select * from
        mlgold.dishes_forecasting_order_history
        where company_id = '{company_id}'
    """
).toPandas()

df_truth.head()

In [ ]:
from dishes_forecasting.paths import PROJECT_DIR
import pandas as pd
forecasting_dir = PROJECT_DIR.parent.parent.parent / "Forecasting"
dishes_file = forecasting_dir / f"dishes_ml_forecast.csv"

df_old_model = pd.read_csv(dishes_file)
df_old_model[df_old_model["company_id"] == company_id]

In [77]:
df_normalization = pd.DataFrame(
    df_old_model.groupby(
        ["year", "week", "run_id"]
    )["dish_ratio"].sum()
).reset_index().rename(
    columns={"dish_ratio": "normalizing_constant"}
)

df_old_model = df_old_model.merge(
    df_normalization,
    on=["year", "week", "run_id"],
    how="left"
)

df_old_model["dish_ratio_normalized"] = df_old_model["dish_ratio"] / df_old_model["normalizing_constant"]

df_old_model= df_old_model.rename(
    columns={
        "variation_id": "product_variation_id",
        "year": "menu_year",
        "week": "menu_week",
        "dish_ratio_normalized": "variation_ratio_predicted",
    }
)

df_old_model = df_old_model.merge(
    df_truth,
    on=["product_variation_id", "menu_year", "menu_week", "company_id"],
    how="left",
).dropna(subset="variation_ratio")

df_old_model["variation_quantity_predicted"] = df_old_model["variation_ratio_predicted"] * df_old_model["total_weekly_qty"]
df_old_model["error"] = df_old_model["variation_quantity_predicted"] - df_old_model["product_variation_quantity"]
df_old_model["abs_error"] = df_old_model["error"].abs()
df_old_model["pct_error"] = df_old_model["error"] / df_old_model["product_variation_quantity"]
df_old_model["abs_pct_error"] = df_old_model["pct_error"].abs()

In [ ]:
df_new_model = connection.sql(
    f"""
        select * from
        mloutputs.ml_dishes_ratios
        where company_id = '{company_id}'
    """
).toPandas()

In [ ]:
df_new_model.columns

In [80]:
import pandas as pd
df_normalization = pd.DataFrame(
    df_new_model.groupby(["menu_year", "menu_week", "created_at"]).variation_ratio_prediction.sum()
).reset_index().rename(
    columns={   "variation_ratio_prediction": "normalizing_constant"}
)

df_new_model = df_new_model.merge(
    df_normalization,
    on=["menu_year", "menu_week", "created_at"],
    how="left"
)

df_new_model["variation_ratio_prediction_normalized"] = df_new_model["variation_ratio_prediction"] / df_new_model["normalizing_constant"]
df_new_model = df_new_model.merge(
    df_truth,
    on=["product_variation_id", "menu_year", "menu_week", "company_id"],
    how="left",
).dropna(subset="variation_ratio")

In [81]:

df_new_model["variation_quantity_predicted"] = df_new_model["variation_ratio_prediction_normalized"] * df_new_model["total_weekly_qty"]
df_new_model["error"] = df_new_model["variation_quantity_predicted"] - df_new_model["product_variation_quantity"]
df_new_model["abs_error"] = df_new_model["error"].abs()
df_new_model["pct_error"] = df_new_model["error"] / df_new_model["product_variation_quantity"]
df_new_model["abs_pct_error"] = df_new_model["pct_error"].abs()

In [ ]:
df_new_model

In [83]:
df_old_model_to_compare = df_old_model[pd.to_datetime(df_old_model["run_timestamp"])>= df_new_model["created_at"].min()]

In [ ]:
df_new_model["abs_error"].describe()

In [ ]:
df_old_model_to_compare["abs_error"].describe()

In [ ]:
df_old_model_to_compare.groupby("run_id")["abs_error"].mean().describe()

In [ ]:
df_new_model.groupby("run_id")["abs_error"].mean().describe()

In [ ]:
df_new_model.groupby("menu_week")["abs_error"].describe()["mean"]

In [ ]:
df_old_model_to_compare.groupby("menu_week")["abs_error"].describe()["mean"]

In [ ]:
df_old_model_to_compare.groupby("run_timestamp")["abs_pct_error"].describe()[["mean", "50%"]]

In [ ]:
df_new_model.groupby("created_at")["abs_pct_error"].describe()[["mean", "50%"]]

In [104]:
df_differences = df_new_model[[
    "menu_year", "menu_week", "product_variation_id", "recipe_name", "variation_ratio_prediction", "variation_quantity_predicted", "created_at", "product_variation_quantity"
]].rename(
    columns={
        "variation_ratio_prediction": "variation_ratio_predicted_new_model",
        "variation_quantity_predicted": "variation_quantity_predicted_new_model",
    }
).merge(
    df_old_model[[
        "product_variation_id", "menu_year", "menu_week", "variation_ratio_predicted", "variation_quantity_predicted"
    ]].rename(
        columns={
            "variation_ratio_predicted": "variation_ratio_predicted_old_model",
            "variation_quantity_predicted": "variation_quantity_predicted_old_model",
        }
    ),
    on=["product_variation_id", "menu_year", "menu_week",],
    how="left",
)



In [105]:
df_differences["differences"] = (df_differences["variation_quantity_predicted_old_model"] - df_differences["variation_quantity_predicted_new_model"])

In [ ]:
df_differences[abs(df_differences["differences"])>50].sample(50)